In [2]:
import fitz
import io
from PIL import Image
import os


pdf_path = 'Kogan_1.pdf'
doc = fitz.open(pdf_path)
page_number = 129  
page = doc.load_page(page_number)
image = page.get_pixmap(matrix=fitz.Matrix(300 / 72, 300 / 72))
temp_filename = "temp_image.jpg"
image.save(temp_filename)

with open(temp_filename, 'rb') as file:
    buffer = io.BytesIO(file.read())

os.remove(temp_filename)

buffer.seek(0)
img = Image.open(buffer)
img.show()

In [16]:
import cv2
import numpy as np

tesseract_path = 'C:/Program Files/Tesseract-OCR/tesseract.exe'
image = cv2.imread('page_132_image_0.jpg')
# Это упрощает обработку изображения.
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# Это помогает уменьшить шум на изображении.
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
# Это выделяет главные элементы изображения.
_, thresh = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY_INV)

cv2.imshow('Threshold Image', thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [17]:
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# Это поможет вам визуально оценить, какие области были выделены.
contour_img = image.copy()
cv2.drawContours(contour_img, contours, -1, (0, 255, 0), 3)
cv2.imshow('Contours', contour_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [18]:
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

custom_config = r'--oem 3 --psm 6'
text = pytesseract.image_to_string(image, config=custom_config, lang='rus+eng+ell')
print(text)


Таблица № 8 (продолжение)
—_———
a | y | t | Р, ата | х | y | i | P, ата ~
|
к
= 96.3 —200 79.62 32.9 84.3 —190 {66.76
12.0 96.4 80.14 34.4 83.1 177.0
42.7 95.9 80.24 36.8 80.8 185.82
16.9 94.8 113.83 38.1 80.5 190.82
16.6 95.1 114.23 40.1 79.5 196.08
ii 94.2 128.31 41.1 76.7 200.69
i 93.5 142.79 441.5 77.7 200.69
20.6 93.0 152.66 43.0 80.0 205.3
== 93.1 152.66 43.5 76.6 205.3
21.8 91.7 177.0 44.8 15.9 210.56
22.0 92.0 177.0 47.0 75.9 210.56
23.6 91.4 186.61 48.6 69.4 221.09
25.0 91.2 205.3 54.4 66.3 223.72
25.7 90.2 205.3 0 0 —185 2.16
26.5 89.3 223.72 3.6 84.0 17.24
27.5 89.0 225.04 5.2 86.6 22.14
0 0 —190 1.18 7A 88.8 31.39
2.7 89.9 17.17 12.6 — 55.4
10.2 93.1 51.19 12.9 89.3 56.85
12.5 93.2 51.19 13.4 90.2 56.98
17.0 92.0 89.36 20.3 88.8 89.36
19.5 91.8 89.36 21.7 88.7 113.18
20.6 90.6 109.89 — 86.7 128.31
21.0 90.7 110.08 30.4 84.8 128.34
22.8 90.4 110.08 30.3 84.8 128.44
24.9 88.9 128.31 41.0 TA 166.47
25.4 88.8 128.31 45.4 70.4 181.61
29.2 84.0 166.87
1
№59 ВОДОРОД—МЕТАН [533]
H,

In [6]:
import fitz  # PyMuPDF
import cv2
import pytesseract
import numpy as np
import os

# Путь к PDF файлу и путь к Tesseract
pdf_path = 'Kogan_1.pdf'
tesseract_path = 'C:/Program Files/Tesseract-OCR/tesseract.exe'
pytesseract.pytesseract.tesseract_cmd = tesseract_path

# Папка для сохранения изображений
images_folder = 'extracted_images'
os.makedirs(images_folder, exist_ok=True)

# Открыть PDF файл
doc = fitz.open(pdf_path)

# Файл для сохранения извлеченного текста
extracted_text_path = 'extracted_text.txt'

# Открыть файл для записи извлеченного текста
with open(extracted_text_path, 'w', encoding='utf-8') as text_file:

    # Перебрать страницы и извлечь изображения
    for page_num in range(129, 135):  # Страницы с 128 по 135
        page = doc.load_page(page_num)
        # Увеличить масштаб для лучшего качества изображения
        pix = page.get_pixmap(matrix=fitz.Matrix(5, 5))
        img_data = pix.tobytes("png")
        img = cv2.imdecode(np.frombuffer(img_data, np.uint8), 1)

        # Сохранить оригинальное изображение
        original_img_path = os.path.join(images_folder, f'page_{page_num + 1}_original.png')
        cv2.imwrite(original_img_path, img)

        # Адаптивная пороговая обработка
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

        # Сохранить обработанное изображение
        processed_img_path = os.path.join(images_folder, f'page_{page_num + 1}_processed.png')
        cv2.imwrite(processed_img_path, thresh)

        # Применение OCR с настроенными параметрами
        custom_config = r'--oem 3 --psm 6'
        text = pytesseract.image_to_string(thresh, config=custom_config, lang='rus+eng')

        # Записать текст в файл
        text_file.write(f'--- Страница {page_num + 1} ---\n{text}\n')

# Закрыть PDF файл
doc.close()


In [8]:
import fitz  # PyMuPDF
import cv2
import pytesseract
import numpy as np
import os

# Путь к PDF файлу и путь к Tesseract
pdf_path = 'Kogan_1.pdf'
tesseract_path = 'C:/Program Files/Tesseract-OCR/tesseract.exe'
pytesseract.pytesseract.tesseract_cmd = tesseract_path

# Папка для сохранения изображений
images_folder = 'extracted_images'
os.makedirs(images_folder, exist_ok=True)

print("Начало обработки PDF файла.")

# Открыть PDF файл
doc = fitz.open(pdf_path)

# Файл для сохранения извлеченного текста
extracted_text_path = 'extracted_text.txt'

def remove_spots(image, kernel_size=(3, 3), area_threshold=100):
    """
    Удаляет мелкие изолированные элементы (кляксы) из изображения.

    :param image: Входное изображение для обработки.
    :param kernel_size: Размер ядра для морфологических операций.
    :param area_threshold: Пороговое значение площади для удаления мелких контуров.
    :return: Обработанное изображение.
    """
    # Морфологическая эрозия и дилатация
    kernel = np.ones(kernel_size, np.uint8)
    erosion = cv2.erode(image, kernel, iterations=1)
    dilation = cv2.dilate(erosion, kernel, iterations=1)

    # Найти контуры на изображении
    contours, _ = cv2.findContours(dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Создать маску для удаления мелких контуров
    mask = np.ones_like(dilation) * 255
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < area_threshold:
            cv2.drawContours(mask, [contour], -1, 0, -1)

    # Применить маску к изображению
    return cv2.bitwise_and(dilation, dilation, mask=mask)


# Открыть файл для записи извлеченного текста
with open(extracted_text_path, 'w', encoding='utf-8') as text_file:

    # Перебрать страницы и извлечь изображения
    for page_num in range(129, 135):  # Страницы с 130 по 135
        print(f"Обработка страницы {page_num + 1}...")
        page = doc.load_page(page_num)
        # Увеличить масштаб для лучшего качества изображения
        pix = page.get_pixmap(matrix=fitz.Matrix(5, 5))
        img_data = pix.tobytes("png")
        img = cv2.imdecode(np.frombuffer(img_data, np.uint8), 1)

        # Сохранить оригинальное изображение
        original_img_path = os.path.join(images_folder, f'page_{page_num + 1}_original.png')
        cv2.imwrite(original_img_path, img)
        print(f"Оригинальное изображение сохранено: {original_img_path}")

        # Адаптивная пороговая обработка
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

        # Проверка, является ли изображение чёрно-белым
        unique_values = np.unique(thresh)
        if len(unique_values) == 2 and set(unique_values) == {0, 255}:
            print("Обработанное изображение чёрно-белое.")
        else:
            print("Обработанное изображение не чёрно-белое.")

        # Сохранить обработанное изображение
        processed_img_path = os.path.join(images_folder, f'page_{page_num + 1}_processed.png')
        cv2.imwrite(processed_img_path, thresh)
        print(f"Обработанное изображение сохранено: {processed_img_path}")

        # Применение OCR с настроенными параметрами
        custom_config = r'--oem 3 --psm 6'
        text = pytesseract.image_to_string(thresh, config=custom_config, lang='rus+eng')
        print(f"Текст с страницы {page_num + 1} извлечен.")

        # Записать текст в файл
        text_file.write(f'--- Страница {page_num + 1} ---\n{text}\n')

# Закрыть PDF файл
doc.close()

print("Обработка PDF файла завершена.")


Начало обработки PDF файла.
Обработка страницы 130...
Оригинальное изображение сохранено: extracted_images\page_130_original.png
Обработанное изображение чёрно-белое.
Обработанное изображение сохранено: extracted_images\page_130_processed.png
Текст с страницы 130 извлечен.
Обработка страницы 131...
Оригинальное изображение сохранено: extracted_images\page_131_original.png
Обработанное изображение чёрно-белое.
Обработанное изображение сохранено: extracted_images\page_131_processed.png
Текст с страницы 131 извлечен.
Обработка страницы 132...
Оригинальное изображение сохранено: extracted_images\page_132_original.png
Обработанное изображение чёрно-белое.
Обработанное изображение сохранено: extracted_images\page_132_processed.png
Текст с страницы 132 извлечен.
Обработка страницы 133...
Оригинальное изображение сохранено: extracted_images\page_133_original.png
Обработанное изображение чёрно-белое.
Обработанное изображение сохранено: extracted_images\page_133_processed.png
Текст с страницы 13